*LLM - GPT Model*

This is a study case of LLMs, developing and training a local GPT model using Open/Free content.

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
import mmap
import random
import pickle
import os
import argparse

In [2]:
print(torch.version.cuda)

12.1


parser = argparse.ArgumentParser(description='This is a demonstration')

parser.add_argument('-batch_size', type=str, required=True, help ='Provide the Batch Size')
args = parser.parse_args()
print(f'batch size: {args.batch_size')

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model_filename = 'jdm_model.pkl'

#Definitions / hyperparameters
batch_size = 64
block_size = 128
max_iters = 200  #3000
learning_rate = 3e-4 #, 3e-4, 1e-3, 1e-4
eval_iters = 100
n_embd = 384
n_head = 8  #8
n_layers = 8  #8 #Normaly same number of decoders
dropout = 0.2

cuda


In [4]:
%%time
start_time = time.time()
zeros = torch.zeros(100,100,100,100)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"{elapsed_time:.8f}")

0.05934048
CPU times: total: 234 ms
Wall time: 59.3 ms


In [5]:
chars = ""
with open('D:/AI_Datasets/LLM/openwebtext/vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

#print(text[:200])


*Tokenizing*
These blocks of code are tokenizing the book, to extract each char and creating a SET

Using enconde/decode functions, we can save a lot of memory since we don't need to

work with all tokens (only with that tokens that are present on the dataset)

#chars = sorted(set(text))  #Tokenizing - Verify the vocabulary (chars) used on this book 
print(chars)
vocab_size = len(chars)
print(vocab_size)

In [6]:
#Creating decoders and encoders to easily convert data from String to int and int to string

string_to_int = { ch:i for i, ch in enumerate(chars)} #Getting the string
int_to_string = { i:ch for i,ch in enumerate(chars)} #Getting the ints
encode = lambda s: [string_to_int[c] for c in s]  #Encoding from string to int
decode = lambda l: ''.join([int_to_string[i] for i in l])  #Decoding from int to string

#Testing
#The expected result should be return a LIST with the index value for each char
encoded_hello = encode('hello')
print(encoded_hello)
decoded_hello = decode(encoded_hello)
print(decoded_hello)

#Creating a tensor with our encoded text
data=torch.tensor(encode(text), dtype=torch.long)
print(data[:200])

#Splitting the dataset (80-20)
n  = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
def get_random_chunk(split):
    filename = "D:/AI_Datasets/LLM/openwebtext/train_split.txt" if split == 'train' else "D:/AI_Datasets/LLM/openwebtext/val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            #Determine the file size and random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            #Seek to random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            #decode the block to a string
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r','')

            #Splitting the dataset into train an test
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data

In [8]:
def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()  #Reducing computation
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    

- B: Batch
- T: Time
- C: Channel

In [10]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)  #key transformation
        q = self.query(x)
        #Compute attention scores
        # Matrix coalculation - query by key (transposed)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 #(B, T, hs) @ (B, hs, T) -> B, T, T
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out
        

In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [12]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
             nn.Linear(n_embd, 4 * n_embd),
             nn.ReLU(),
             nn.Linear(4 * n_embd, n_embd),
             nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [13]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)   #Self Attentiion
        x = self.ln1(x + y)  #Linear Normalization
        y = self.ffwd(x)  #Feed Forward
        x = self.ln2(x + y) #Linear Normalization
        return x

In [14]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        #Blocs
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layers)])  #Creating the decoding blocks
        self.ln_f = nn.LayerNorm(n_embd)   #Final normalization Layer
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, index, targets=None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  #(T, C)
        x = tok_emb + pos_emb    #(B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indexes in the current context
        for _ in range(max_new_tokens):
            #Getting predictions
            logits, loss = self.forward(index)
            #Focus only on the last time step
            logits = logits[:, -1, :]
            #Apply softmax to get probs
            probs = F.softmax(logits, dim=-1)
            #Sample from distrib
            index_next = torch.multinomial(probs, num_samples=1)
            #Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)  #(B, T+1)
        return index
            

model = GPTLanguageModel(vocab_size)
m = model.to(device)

if os.path.exists(model_filename):
    print("Loading model parameters...")
    with open(model_filename, 'rb') as f:
        model = pickle.load(f)
    print("lodaded")
    m = model.to(device)

In [15]:
#Create a PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss {losses['train']:.3f}, val loss {losses['val']:.3f}")
    
    
    xb, yb = get_batch('train')

    #evaluate
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open(model_filename, 'wb') as f:
    pickle.dump(model, f)
print('Model saved')

step: 0, train loss 10.456, val loss 10.443
step: 100, train loss 2.452, val loss 2.393
3.1472861766815186
Model saved


context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)